In [ ]:
!pip install PyPDF2 pydata-google-auth groq langchain

In [ ]:
from groq import Groq
from langchain.prompts import PromptTemplate
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from PyPDF2 import PdfReader
import datetime
import base64
import os
import pydata_google_auth
from google.colab import files

In [ ]:
import os

API_KEY = os.getenv("API_KEY")


In [ ]:
SCOPES = ['https://www.googleapis.com/auth/calendar', 'https://www.googleapis.com/auth/gmail.send']

In [ ]:
import google.auth
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import os
import json

def authenticate_google():
    creds = None
    # Check if token.json exists (previously saved credentials)
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
        print("Token File Exists!!")
    # If there are no (valid) credentials, request new ones
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # Use InstalledAppFlow for authentication
            # The credentials_gcp.json file needs to be uploaded to the Colab environment
            from google.colab import files
            uploaded = files.upload()
            # Assuming the file is named 'credentials_gcp.json'
            # Replace 'credentials_gcp.json' with the actual filename if different
            file_name = next(iter(uploaded))

            flow = InstalledAppFlow.from_client_secrets_file(
                '/content/credentials_gcp.json', SCOPES
            )
            # Generate the authorization URL
            flow.redirect_uri = "https://poditivity.com"
            auth_url, _ = flow.authorization_url(prompt="consent")
            print(f"Please visit this URL to authorize this application: {auth_url}")
            # Prompt the user to enter the authorization code from the redirected URL
            code = input("Enter the authorization code: ")
            # Fetch the token using the authorization code
            flow.fetch_token(code=code)
            creds = flow.credentials
        # Save the credentials for the next run
        with open("token.json", "w") as token_file:
            json.dump({
                "token": creds.token,
                "refresh_token": creds.refresh_token,
                "token_uri": creds.token_uri,
                "client_id": creds.client_id,
                "client_secret": creds.client_secret,
                "scopes": creds.scopes
            }, token_file)
    return creds

In [ ]:
#authenticate_google()

In [ ]:
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = "".join(page.extract_text() for page in reader.pages)
    return text

In [ ]:
def assess_suitability(job_description, resume):
    # Define a prompt template
    prompt_template = PromptTemplate(
        input_variables=["resume", "job_description"],
        template=(
            "Consider yourself as an experienced talent acquisition specialist. Given the job description and the candidate's resume below, "
            "determine if the candidate is suitable for the role.\n\n"
            "Job Description:\n{job_description}\n\n"
            "Candidate Resume:\n{resume}\n\n"
            "Respond STRICTLY with 'YES' or 'NO'. 'YES' if the candidate is suitable, otherwise 'NO'."
        ),
    )

    query = prompt_template.format(job_description=job_description, resume=resume)
    client = Groq(api_key=api_key)

    response = client.chat.completions.create(
        messages=[{
                "role": "system",
                "content": "You are a talent acquisition specialist, analyze the resume against the job description provided."
            },
            {
                "role": "user",
                "content": query
            }
        ],
        model="llama3-8b-8192",
    )
    print(response)

    return response.choices[0].message.content

def send_email(service, to_email, subject, body):
    message = {
        "raw": base64.urlsafe_b64encode(
            f"To: {to_email}\nSubject: {subject}\n\n{body}".encode("utf-8")
        ).decode("utf-8")
    }
    try:
        service.users().messages().send(userId="me", body=message).execute()
    except HttpError as error:
        print(f"An error occurred: {error}")

In [ ]:
def schedule_interview(service, candidate_email, candidate_name):
    now = datetime.datetime.utcnow()
    start_time = now + datetime.timedelta(days=1, hours=3)  # Set for next day
    end_time = start_time + datetime.timedelta(minutes=30)

    event = {
        "summary": f'Interview with {candidate_name}',
        "location": "Google Meet",
        "description": "Discussion regarding the job application.",
        "start": {"dateTime": start_time.isoformat() + "Z", "timeZone": "UTC"},
        "end": {"dateTime": end_time.isoformat() + "Z", "timeZone": "UTC"},
        "conferenceData": {
            "createRequest": {
                "requestId": "sample123",
                "conferenceSolutionKey": {"type": "hangoutsMeet"},
            }
        },
        "attendees": [{"email": candidate_email}],
    }
    try:
        event_result = service.events().insert(
            calendarId="primary",
            body=event,
            conferenceDataVersion=1,
        ).execute()

        return event_result.get("hangoutLink")
    except HttpError as error:
        print(f"An error occurred: {error}")
        return None

In [ ]:
def schedule_interview(service, candidate_email, candidate_name):
    now = datetime.datetime.utcnow()
    start_time = now + datetime.timedelta(days=1, hours=3)  # Set for next day
    end_time = start_time + datetime.timedelta(minutes=30)

    event = {
        "summary": f'Interview with {candidate_name}',
        "location": "Google Meet",
        "description": "Discussion regarding the job application.",
        "start": {"dateTime": start_time.isoformat() + "Z", "timeZone": "UTC"},
        "end": {"dateTime": end_time.isoformat() + "Z", "timeZone": "UTC"},
        "conferenceData": {
            "createRequest": {
                "requestId": "sample123",
                "conferenceSolutionKey": {"type": "hangoutsMeet"},
            }
        },
        "attendees": [{"email": candidate_email}],
    }
    try:
        event_result = service.events().insert(
            calendarId="primary",
            body=event,
            conferenceDataVersion=1,
        ).execute()

        return event_result.get("hangoutLink")
    except HttpError as error:
        print(f"An error occurred: {error}")
        return None

In [ ]:
def resume_screening(job_description_pdf, resume_pdf, candidate_email, candidate_name):
    creds = authenticate_google()
    gmail_service = build("gmail", "v1", credentials=creds)
    calendar_service = build("calendar", "v3", credentials=creds)

    job_description = extract_text_from_pdf(job_description_pdf)
    resume = extract_text_from_pdf(resume_pdf)

    decision = assess_suitability(job_description, resume)

    print(decision)

    if decision == "NO":
        send_email(
            gmail_service,
            to_email=candidate_email,
            subject="Application Update",
            body=f"Dear, {candidate_name}\n\n"
            "Thank you for applying. Unfortunately, your skills do not match the role requirements at this time. We wish you the best of luck in your career!",
        )
    elif decision == "YES":
        meet_link = schedule_interview(calendar_service, candidate_email, candidate_name)
        if meet_link:
            send_email(
                gmail_service,
                to_email=candidate_email,
                subject="Interview Invitation",
                body=(
                    f"Congratulations {candidate_name}!\n\n"
                    "You've been shortlisted for the role.\n"
                    f"Please join us for an interview at the following link: {meet_link}"
                ),
            )

In [ ]:
from google.colab import files # Import the files object

print("Upload job description PDF")
job_desc_file = files.upload()
job_desc_path = next(iter(job_desc_file.keys()))

In [ ]:
print("Upload candidate resume PDF")
resume_file = files.upload()
resume_path = next(iter(resume_file.keys()))
candidate_email = input("Enter candidate's email: ")
candidate_name=input("Enter candidate's name: ")

In [ ]:
resume_screening(job_desc_path, resume_path, candidate_email,candidate_name)